In [9]:
sources = [
    "https://www.wattpad.com/627607959-matar-waye-1-10",
    "https://www.wattpad.com/783423722-ummulkhair-%F0%9F%91%A9%E2%80%8D%F0%9F%91%A7%F0%9F%92%9D%F0%9F%91%A8%E2%80%8D%F0%9F%91%A7-part-1",
    "https://www.wattpad.com/435279154-khairat-1",
    "https://www.wattpad.com/438014700-sanadin-ki-rayuwarmu-da-suhailat",
    "https://www.wattpad.com/769019871-rashin-dace-introduction",
    "https://www.wattpad.com/664888317-auren-dole-2015-part-1",
    "https://www.wattpad.com/606963558-zuciyar-mutum-birninsa-1~fitila",
    "https://www.wattpad.com/308506210-yar-kurma-comple%E2%88%9A-yar-kurma",
    "https://www.wattpad.com/459645599-nadamar-rayuwata-complt-babi-na-daya",
    "https://www.wattpad.com/635667144-kundin-haske%F0%9F%92%A1",
    "https://www.wattpad.com/453600216-idan-ka-daka-ta-bado-complt-babi-na-%27daya",
    "https://www.wattpad.com/559513141-jasmine-baturiyya-ce-%F0%9F%8C%B7jasmine%F0%9F%8C%B7",
    "https://www.wattpad.com/392145778-gimbiya-zajlat-cmplt-gimbiya-zajlat-babi-na-%27daya",
    "https://www.wattpad.com/804845120-bakar-zargi%F0%9F%96%A4%F0%9F%96%A4a-hausa-love-story%F0%9F%96%A4%F0%9F%96%A4-completed",
    "https://www.wattpad.com/556381904-sarauniya-jidda-na-aunty-fyn-love-story-babi-na",
    "https://www.wattpad.com/790747730-wadata-one",
    "https://www.wattpad.com/714141565-abadan",
    "https://www.wattpad.com/660395766-dabaibayi-samira",
    "https://www.wattpad.com/595953677-maraicin-%27ya-mace-gabatarwa",
    "https://www.wattpad.com/746258113-hauwa",
    "https://www.wattpad.com/425157222-akwai-lokaci-babi-na-daya",
    "https://www.wattpad.com/762844517-uwa-ta-gari-prolong",
    "https://www.wattpad.com/729342139-amsoshin-tambayoyinku-tambayoyi-da-amsoshi",
    "https://www.wattpad.com/764388392-kambun-sadaukantaka",
    "https://www.wattpad.com/776122898-yuhaina%F0%9F%92%96-notice",
    "https://www.wattpad.com/482828599-kaine-muradina-gabatarwa",
    "https://www.wattpad.com/754111276-abuncin-wani-page-one",
    "https://www.wattpad.com/751834446-zabina-part-2-complete-hausanovel",
    "https://www.wattpad.com/606671070-nadamar-rayuwa-wasikar-farko",
    "https://www.wattpad.com/781711723-rashin-uba-shafi-na-1",
    "https://www.wattpad.com/619040718-nainah-1-5"
]

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, ElementNotInteractableException, NoSuchElementException
from bs4 import BeautifulSoup as bs
from nltk import tokenize
import time

In [3]:
chrome_path = 'C:\\Users\\AkintundeOladipo\\Downloads\\chromedriver_win32\\chromedriver.exe'
comment_xpath = "/html/body/div[4]/div/main/article/footer/div[1]/div[3]/div[1]/div/div/div/section/div/div[3]/button/span[1]/span[2]"
chapter_xpath = "/html/body/div[4]/div/div[1]/div/div[1]/div/ul/li[{}]/a/div"

def init_driver():
    
    driver = webdriver.Chrome(chrome_path)
    
    # define a generic wait to be used throughout
    driver.wait = WebDriverWait(driver, 5)
    
    # dataframe for a possible sentiment analysis
    # storage = pd.DataFrame(columns=["source", "no_chapters", "chapter_titles", "tags", "text", "interactions", "comments"])
    
    return driver

def show_all_comments(comment_xpath):
    """
    Shows all comments existing on storage page
    """
    try:
        driver.find_element_by_xpath(comment_xpath).click()
        driver.implicitly_wait(10)
        # driver.find_element_by_tag_name('html').send_keys(Keys.END)
        scroll(driver, 5)
    except ElementClickInterceptedException:
        """
        if element is still covered by another element because page
        is loading, wait
        """
        driver.wait.until(EC.visibility_of_element_located((By.XPATH, comment_xpath)))
    except (ElementNotInteractableException, NoSuchElementException):
        """
        if element does not exist or is now hidden, exit. 
        """
        return False
    
    return True

def remove_emojis(input_string):
    """
    This func removes emojis from text
    """
    return input_string.encode("ascii", "ignore").decode("ascii")

def close_driver(driver):
    
    driver.close()
    
def to_txt(sentences, file):
    """
    This func writes a list of sentences into a file line by line.
    """
    sentence_split = filter(None, sentences.split("."))
    
    for s in sentence_split:
        #print(s.strip())
        file.write(s.strip() + "\n")
        
def init_storage():
    """
    This func creates unique dataframes for stories
    collected from different sources
    """
    storage = pd.DataFrame(columns=["source","chapter_titles", 
                                    "tags", "text", 
                                    "interactions", "comments"])
    
    # initialize empty list to hold entities to be collected
    comments = list()
    
    return storage, comments

def scroll(driver, timeout):
    """
    This func scrolls to the bottom of the Hausa
    Writers Series page.
    """
    
    scroll_pause_timeout = timeout
    
    # get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        
        #scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # wait for page to load
        time.sleep(scroll_pause_timeout)
        
        # calculate new scroll height & compare with previous
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break # bottom has been reached
        
        last_height = new_height
    
def download_stories(driver, sources):
    
    interaction_types = ["reads", "likes", "comments"]
    
    for i, source in enumerate(sources):
        
        print(f"Starting Source {i}")
        
        filename = "../data/wattpad/Source_{0}.txt"
        file = open(filename.format(i), "w+", encoding="utf-8")
        
        # Source must be link of first chapter of story
        driver.get(source)
        
        # initializes dataframe and list to store data
        #storage, comments = init_storage()
        
        time.sleep(2) 
        
        # finds Table of Contents on page
        driver.find_element_by_css_selector('.btn.btn-toc.dropdown-toggle').click()
        
        # grabs number of chapters and their titles
        options = driver.find_element_by_css_selector(".dropdown-menu.pull-left > ul")
        chapter_titles = options.text.split("\n")
        
        # write the chapter titles 
        #storage.loc[:, "chapter_titles"] = chapter_titles
        n_chapters = len(chapter_titles) 
        
        time.sleep(2)
        
        driver.find_element_by_css_selector('.btn.btn-toc.dropdown-toggle').click()
        
        # loops over the chapters collecting entities into the dataframe and text file.
        # loop starts from 1 because 
        for chapter in range(1, n_chapters+1):
            # print(chapter)
            # driver.refresh()
            #body_of_story = ""
            # initialize list to hold texts
            #text = list()
            #comments = list()
            
            if chapter != 1:
                # wait for dropdown to show before selecting 
                driver.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".btn.btn-toc.dropdown-toggle")))
                driver.find_element_by_css_selector('.btn.btn-toc.dropdown-toggle').click()

                # wait for chapter in focus to show before selecting
                driver.wait.until(EC.presence_of_element_located((By.XPATH, chapter_xpath.format(chapter))))
                driver.find_element_by_xpath(chapter_xpath.format(chapter)).click()
            
            # go to bottom of page
            time.sleep(2)
            
            #html = driver.find_element_by_tag_name('html')
            #html.send_keys(Keys.END)
            
            scroll(driver, 2)
            
            time.sleep(2)
            
            # Show all comments            
            while show_all_comments(comment_xpath):
                
                time.sleep(2)
                driver.find_element_by_tag_name('html').send_keys(Keys.END)
                time.sleep(2)

                show_all_comments(comment_xpath)
            
            #time.sleep(5)
            try:
                content = driver.page_source
            except:
                continue
                
            soup = bs(content)
            
            #storage.loc[chapter-1, "interactions"] = str({k:v for k, v in zip(interaction_types, 
                                                                                #soup.findAll('div', attrs={'class': "story-stats"
                                                                                                          #})[0].text.split())})
            #print(str({k:v for k, v in zip(interaction_types, 
                                           #soup.findAll('div', attrs={'class': "story-stats"
                                                                     #})[0].text.split())}))
            to_txt(remove_emojis(soup.title.text), file)
            
            # collect both title and body of story 
            for j, options in enumerate(soup.findAll('p', {"data-p-id": True})):

                # convert to sentences and add it to our list of sentences
                to_txt(remove_emojis(options.text), file)
                #body_of_story += options.text + "\n"
                
            #storage.loc[chapter-1, "text"] = body_of_story
                    
            # loop over comment class and append to list
            file.write(" - Comments - \n")
            for comment in soup.findAll('div', attrs={'class': "comment"}):
                
                to_txt(comment.find('pre').text, file)
                
            # write collected sentences into file
            #to_txt(text, file)    
            #to_txt(comments, file)
            
            # print(comments)
            # write comments to dataframe as str    
            #storage.loc[chapter-1, "comments"] = str(comments)
                
            # when you get to the last chapter, write out the tags and source for the story.
            # These do not change with chapter. 
            #if chapter == n_chapters:
                
                #tags.extend([tag.text for tag in soul.findAll('p', attrs={'class': "item-tags"})])
                #storage.loc[:, "tags"] = [tag.text for tag in soup.findAll('p', attrs={'class': "item-tags"})] * storage.shape[0]
                
                # write the source on every row 
                #storage.loc[:, "source"] = source 
                
            time.sleep(2)
                
        #storage.to_csv(f"Source_{i}.csv")
        file.close()
        print(f"Finished Source no. {i}")

In [11]:
if __name__ == "__main__":
    
    driver = init_driver()
    
    driver.implicitly_wait(10)
    
    download_stories(driver, sources)
    
    print("All done!")
    
    close_driver(driver)

Starting Source 0
Finished Source no. 0
Starting Source 1
Finished Source no. 1
Starting Source 2
Finished Source no. 2
Starting Source 3
Finished Source no. 3
Starting Source 4
Finished Source no. 4
Starting Source 5
Finished Source no. 5
Starting Source 6
Finished Source no. 6
Starting Source 7
Finished Source no. 7
Starting Source 8
Finished Source no. 8
Starting Source 9
Finished Source no. 9
Starting Source 10
Finished Source no. 10
Starting Source 11
Finished Source no. 11
Starting Source 12
Finished Source no. 12
Starting Source 13
Finished Source no. 13
Starting Source 14
Finished Source no. 14
Starting Source 15
Finished Source no. 15
Starting Source 16
Finished Source no. 16
All done!
